# SPDE simulation on sphere

The aim of this tutorial is to show how to use gstlearn to simulate the solution of 

$$(\kappa^2-\Delta_{\mathcal{S}_R})^{\alpha/2}Z = \sigma\mathcal{W}$$

on the sphere $\mathcal{S}_R$ of radius $R$.

- $\Delta_{{\mathcal{S}_R}}$ is the Laplace-Beltrami operator, i.e, it acts on each point of the sphere as the usual Laplacian on the tangent plane at this point. 

- $\kappa$ is the inverse of the scale parameter

- $\alpha \geq 2$ is an integer describing the smoothness of the solution.

- $\mathcal{W}$ is a Gaussian white-noise suitably normalized such as $\sigma^2$ is the variance of the solution.

In this notebook, we will define the covariance of Matérn on the sphere, as the covariance of the solution of this SPDE (other extensions of the Matérn function are possible). By analogy with the Euclidian case, its smoothness parameter will be defined by $\nu = \alpha -1$. To compute the covariance function with respect on the geodetic distance, one have to use a decomposition on the Legendre polynomial (see below).

In [ ]:
import gstlearn as gl

import numpy as np           
from numpy import pi, cos, sin

# Plot packages 

import gstlearn.plot as gp
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import IPython

### Parametrization

In [ ]:
#Sphere radius
    
R = 10.

gl.variety_define(1,R)

# Smoothness parameter
nu = 1 

#Scale parameter 
#For convenien,ce, it is defined as the proportion of the radius

ratioRange = 0.2  
scale = R * ratioRange
# sill 
sill = 2. 


### Meshing

In [ ]:
mesh = gl.MeshSpherical()
err = mesh.reset(None,None,triswitch = "-r5",verbose=False)

In [ ]:
model = gl.Model.createFromParam(type=gl.ECov.BESSEL_K,
                                 range = scale,
                                 sill = sill,
                                 param=nu,
                                 flagRange= False)
model

### Precision matrix

In [ ]:
Q = gl.PrecisionOp(mesh,model)

### Simulation 

In [ ]:
result = np.array(Q.simulate()[0])

### Display the realization
 

In [ ]:
X = mesh.getCoordinates(0)
Y = mesh.getCoordinates(1)

coords = np.array(gl.util_convert_longlat(X,Y,dilate=0.99))

meshes = np.array(mesh.getMeshes()).reshape([mesh.getNMeshes(),3])-1

cscale = [[0.0, '#313695'],
 [0.07692307692307693, '#3a67af'],
 [0.15384615384615385, '#5994c5'],
 [0.23076923076923078, '#84bbd8'],
 [0.3076923076923077, '#afdbea'],
 [0.38461538461538464, '#d8eff5'],
 [0.46153846153846156, '#d6ffe1'],
 [0.5384615384615384, '#fef4ac'],
 [0.6153846153846154, '#fed987'],
 [0.6923076923076923, '#fdb264'],
 [0.7692307692307693, '#f78249'],
 [0.8461538461538461, '#e75435'],
 [0.9230769230769231, '#cc2727'],
 [1.0, '#a50026']]


fig = go.Figure(data=[
    go.Mesh3d(
        x=coords[0,:],
        y=coords[1,:],
        z=coords[2,:],
        colorbar_title='z',
        colorscale=cscale,
        # Intensity of each vertex, which will be interpolated and color-coded
        intensity=result,
        # i, j and k give the vertices of triangles
        # here we represent the 4 triangles of the tetrahedron surface
        i=meshes[:,0],
        j=meshes[:,1],
        k=meshes[:,2],
        name='y',
        showscale=True
    )
])

fig.show()

### Compute covariance (of discretized solution)

We use the fact that $\Sigma = Q^{-1}$ and solve $Qx = e_j$ for an arbitrary index $j$.

In [ ]:
long = np.array(mesh.getCoordinates(0))
lat = np.array(mesh.getCoordinates(1))
indClose = np.where(np.abs(lat)<=.5)[0]
ind0 = int(indClose[0])
##
covDiscr = np.array(Q.evalCov(ind0))
##
covDiscrClose = covDiscr[indClose]
deltaLong = np.abs(long[indClose]-long[ind0])/180 * np.pi
deltaLong[deltaLong> np.pi] = 2 * np.pi - deltaLong[deltaLong> np.pi] 
covDiscrClose = covDiscrClose[np.argsort(deltaLong)]
deltaLong = R * np.sort(deltaLong)

### Variogram of the realization

The empirical variogram is computed by using the great-circle distance.

In [ ]:
npas = 50 # number of discretization points
dpas = 0.04 # lag with respect to the unit sphere (it will be multiplied
# by R in the creation of the VarioParam.

nsample = 4000
#sub-sampling to reduce computational burden

ind = np.random.choice(mesh.getNApices(),size=nsample,replace=False)

#Creation of the db

dbdat = gl.Db.create()
dbdat["x"] = np.array(X)[ind]
dbdat["y"] = np.array(Y)[ind]
dbdat["z"] = np.array(result)[ind]
dbdat.setLocators(["x","y"],gl.ELoc.X)
dbdat.setLocators(["z"],gl.ELoc.Z)

#Variogram 

vp = gl.VarioParam.createOmniDirection(npas=npas,dpas=dpas * R)
vario = gl.Vario.create(vp,dbdat)
vario.compute(gl.ECalcVario.VARIOGRAM)

### Theoretical covariance function

The covariance between two points with great-circle distance $d$  on the sphere of radius $R$ is given by
$$C(d) = \frac{\sigma^2}{\sum_{i=0}^\infty f(i)}\sum_{i=0}^\infty f(i) P_i\left(\cos \frac{d}{R}\right)$$

where the $P_i$'s  are the Legendre polynomials computed with the following reccurence formula

$$P_0(x) = 1.$$

$$P_1(x) = x$$

$$P_{n+1}(x)=\frac{(2n+1)xP_n(x) - n P_{n-1}(x)}{n+1}$$

For $n\geq 0$, $$f(n) = \frac{2 (n + 1)}{ (R^2\kappa^2 + n ( n + 1))^\alpha}$$

For numerical computations, the sums are truncated at **N**.

### Evaluation 

In [ ]:
ndisc = 100 # number of discretization steps for the covariance 
N = 50 # size of the decomposition

hnorm = np.linspace(0,npas * dpas,ndisc)
ax=gp.vario(vario,end_plot=False,label="vario1")
a=model.getCova(0).getCova()
uu = [a.evalCovOnSphere(i,scale/R,N) for i in hnorm]
uu = np.array(uu)/uu[0] #Renormation
ax = plt.plot(R * hnorm, sill - sill * uu,label = "Theoretical covariance")
plt.plot(deltaLong,sill - covDiscrClose,"--",label = "Discretized model",)
ax = plt.legend()